## Lab 3a(part equivalent to Lab 2a)

In [1]:
#setting up
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine, text

import pandas as pd
import matplotlib.pyplot as plt

host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "root_user"
pwd = "Eutocomfome,meuamigo!"
db_name = "northwind"
dst_db_name = "northwind_dw2"

In [2]:
def readSqlQuery(sql_query, db):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    with sqlEngine.connect() as conn:
        return pd.read_sql(text(sql_query), conn);

def setSqlQuery(sql_query, db):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    with sqlEngine.connect() as conn:
        conn.execute(text(sql_query))


In [3]:
# create destination database
setSqlQuery(f"CREATE DATABASE IF NOT EXISTS {dst_db_name}",db_name)

In [4]:
sql_create_datawarehouse = [
"""
CREATE TABLE IF NOT EXISTS `dim_customers` (
  `customer_key` int NOT NULL AUTO_INCREMENT,
  `company` varchar(50) DEFAULT NULL,
  `last_name` varchar(50) DEFAULT NULL,
  `first_name` varchar(50) DEFAULT NULL,
  `job_title` varchar(50) DEFAULT NULL,
  `business_phone` varchar(25) DEFAULT NULL,
  `fax_number` varchar(25) DEFAULT NULL,
  `address` longtext,
  `city` varchar(50) DEFAULT NULL,
  `state_province` varchar(50) DEFAULT NULL,
  `zip_postal_code` varchar(15) DEFAULT NULL,
  `country_region` varchar(50) DEFAULT NULL,
  PRIMARY KEY (`customer_key`),
  KEY `city` (`city`),
  KEY `company` (`company`),
  KEY `first_name` (`first_name`),
  KEY `last_name` (`last_name`),
  KEY `zip_postal_code` (`zip_postal_code`),
  KEY `state_province` (`state_province`)
) ENGINE=InnoDB AUTO_INCREMENT=30 DEFAULT CHARSET=utf8mb4;
""",
"""
CREATE TABLE IF NOT EXISTS `dim_employees` (
  `employee_key` int NOT NULL AUTO_INCREMENT,
  `company` varchar(50) DEFAULT NULL,
  `last_name` varchar(50) DEFAULT NULL,
  `first_name` varchar(50) DEFAULT NULL,
  `email_address` varchar(50) DEFAULT NULL,
  `job_title` varchar(50) DEFAULT NULL,
  `business_phone` varchar(25) DEFAULT NULL,
  `home_phone` varchar(25) DEFAULT NULL,
  `fax_number` varchar(25) DEFAULT NULL,
  `address` longtext,
  `city` varchar(50) DEFAULT NULL,
  `state_province` varchar(50) DEFAULT NULL,
  `zip_postal_code` varchar(15) DEFAULT NULL,
  `country_region` varchar(50) DEFAULT NULL,
  `web_page` longtext,
  PRIMARY KEY (`employee_key`),
  KEY `city` (`city`),
  KEY `company` (`company`),
  KEY `first_name` (`first_name`),
  KEY `last_name` (`last_name`),
  KEY `zip_postal_code` (`zip_postal_code`),
  KEY `state_province` (`state_province`)
) ENGINE=InnoDB AUTO_INCREMENT=10 DEFAULT CHARSET=utf8mb4;
""",
"""
CREATE TABLE IF NOT EXISTS `dim_products` (
  `product_key` int NOT NULL AUTO_INCREMENT,
  `product_code` varchar(25) DEFAULT NULL,
  `product_name` varchar(50) DEFAULT NULL,
  `standard_cost` decimal(19,4) DEFAULT '0.0000',
  `list_price` decimal(19,4) NOT NULL DEFAULT '0.0000',
  `reorder_level` int DEFAULT NULL,
  `target_level` int DEFAULT NULL,
  `quantity_per_unit` varchar(50) DEFAULT NULL,
  `discontinued` tinyint(1) NOT NULL DEFAULT '0',
  `minimum_reorder_quantity` int DEFAULT NULL,
  `category` varchar(50) DEFAULT NULL,
  PRIMARY KEY (`product_key`),
  KEY `product_code` (`product_code`)
) ENGINE=InnoDB AUTO_INCREMENT=100 DEFAULT CHARSET=utf8mb4;
""",
"""
CREATE TABLE IF NOT EXISTS `dim_shippers` (
  `shipper_key` int NOT NULL AUTO_INCREMENT,
  `company` varchar(50) DEFAULT NULL,
  `address` longtext,
  `city` varchar(50) DEFAULT NULL,
  `state_province` varchar(50) DEFAULT NULL,
  `zip_postal_code` varchar(15) DEFAULT NULL,
  `country_region` varchar(50) DEFAULT NULL,
  PRIMARY KEY (`shipper_key`),
  KEY `city` (`city`),
  KEY `company` (`company`),
  KEY `zip_postal_code` (`zip_postal_code`),
  KEY `state_province` (`state_province`)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8mb4;
""",
"""
CREATE TABLE IF NOT EXISTS `dim_suppliers` (
  `supplier_key` int NOT NULL AUTO_INCREMENT,
  `company` varchar(50) DEFAULT NULL,
  `last_name` varchar(50) DEFAULT NULL,
  `first_name` varchar(50) DEFAULT NULL,
  `job_title` varchar(50) DEFAULT NULL,
  PRIMARY KEY (`supplier_key`),
  KEY `company` (`company`),
  KEY `first_name` (`first_name`),
  KEY `last_name` (`last_name`)
) ENGINE=InnoDB AUTO_INCREMENT=11 DEFAULT CHARSET=utf8mb4;
""",
"""CREATE TABLE IF NOT EXISTS `fact_orders`(
-- orders
	`fact_order_key` int NOT NULL AUTO_INCREMENT,
	`order_key` int NOT NULL,
	`employee_key` int DEFAULT NULL,
	`customer_key` int DEFAULT NULL,
	`order_date` datetime DEFAULT NULL,
	`shipped_date` datetime DEFAULT NULL,
	`shipper_key` int DEFAULT NULL,
	`ship_name` varchar(50) DEFAULT NULL,
	`ship_address` longtext DEFAULT NULL,
	`ship_city` varchar(50) DEFAULT NULL,
	`ship_state_province` varchar(50) DEFAULT NULL,
	`ship_zip_postal_code` varchar(50) DEFAULT NULL,
	`ship_country_region` varchar(50) DEFAULT NULL,
	`shipping_fee` decimal(19,4) DEFAULT NULL,
    `taxes` decimal(19,4) DEFAULT NULL,
	`payment_type` varchar(50) DEFAULT NULL,
	`paid_date` datetime DEFAULT NULL,
	`tax_rate` double DEFAULT NULL,
-- order details
	`product_key` int DEFAULT NULL,	
	`quantity` decimal(18,4) DEFAULT NULL,
	`unit_price` decimal(18,4) DEFAULT NULL,
	`discount` double DEFAULT NULL,
-- orders status
	`order_status` varchar(50) DEFAULT NULL,
-- order details status
	`order_details_status` varchar(50) DEFAULT NULL,
    PRIMARY KEY `fact_order_key` (`fact_order_key`),
	KEY `order_key` (`order_key`),
    KEY `customer_key` (`customer_key`),
    KEY `employee_key` (`employee_key`),
    KEY `product_key` (`product_key`),
    KEY `shipper_key` (`shipper_key`),
    KEY `ship_zip_postal_code` (`ship_zip_postal_code`)
) ENGINE=InnoDB AUTO_INCREMENT=11 DEFAULT CHARSET=utf8mb4;"""
]

In [5]:
for thing in sql_create_datawarehouse: setSqlQuery(thing,dst_db_name)

In [6]:
#setSqlQuery(f"DROP DATABASE IF EXISTS {dst_db_name}",dst_db_name)

## Lab 3b(part equivalent to Lab 2b)

In [7]:
#low hanging fruit first
customers = readSqlQuery("SELECT * FROM customers", db_name)
employees = readSqlQuery("SELECT * FROM employees", db_name)
products = readSqlQuery("SELECT * FROM products", db_name)
shippers = readSqlQuery("SELECT * FROM shippers", db_name)

In [8]:
def set_dataframe(df,db,table_name, db_operation,pk_column):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    with sqlEngine.connect() as conn:
        if db_operation == "insert": 
            df.to_sql(table_name, con=conn, index=False, if_exists='replace')
            conn.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column})"))
        elif db_operation == "update": 
            df.to_sql(table_name, con=conn, index=False, if_exists='append')

In [9]:
customers = customers.rename(columns={"id":"customer_key"})
employees = employees.rename(columns={"id":"employee_key"})
products = products.rename(columns={"id":"product_key"})
shippers = shippers.rename(columns={"id":"shipper_key"})

#use lab2 query to determine what columns should be selected
lab2_dim_customer_list = ["customer_key",
"company",
"last_name",
"first_name",
"job_title",
"business_phone",
"fax_number",
"address",
"city",
"state_province",
"zip_postal_code",
"country_region"]
lab2_dim_employees_list = ["employee_key",
"company",
"last_name",
"first_name",
"email_address",
"job_title",
"business_phone",
"home_phone",
"fax_number",
"address",
"city",
"state_province",
"zip_postal_code",
"country_region",
"web_page"]
lab2_dim_products_list = ["product_key",
"product_code",
"product_name",
"standard_cost",
"list_price",
"reorder_level",
"target_level",
"quantity_per_unit",
"discontinued",
"minimum_reorder_quantity",
"category"]
lab2_dim_shippers_list = ["shipper_key",
"company",
"address",
"city",
"state_province",
"zip_postal_code",
"country_region"]


dim_customers = customers[lab2_dim_customer_list]
dim_employees = employees[lab2_dim_employees_list]
dim_products = products[lab2_dim_products_list]
dim_shippers = shippers[lab2_dim_shippers_list]



In [10]:
#obs: the primary key was set before so I used update
set_dataframe(dim_customers,dst_db_name, "dim_customers", "update","customer_key")
set_dataframe(dim_employees,dst_db_name, "dim_employees", "update","employee_key")
set_dataframe(dim_products,dst_db_name, "dim_products", "update","product_key")
set_dataframe(dim_shippers,dst_db_name, "dim_shippers", "update","shipper_key")


In [11]:
#validate insertions
#readSqlQuery("SELECT * FROM dim_customers", dst_db_name).head(2)
#readSqlQuery("SELECT * FROM dim_employees", dst_db_name).head(2)
#readSqlQuery("SELECT * FROM dim_products", dst_db_name).head(2)
#readSqlQuery("SELECT * FROM dim_shippers", dst_db_name).head(2)

In [12]:
orders = readSqlQuery("SELECT * FROM orders", db_name)
order_details = readSqlQuery("SELECT * FROM order_details", db_name)
orders_status = readSqlQuery("SELECT * FROM orders_status", db_name)
order_details_status = readSqlQuery("SELECT * FROM order_details_status", db_name)

orders = orders.rename(columns={"id":"order_key",
                                "employee_id":"employee_key",
                                "customer_id":"customer_key",
                                "shipper_id":"shipper_key"
                               })
order_details = order_details.rename(columns={"product_id":"product_key"})
orders_status = orders_status.rename(columns={"status_name":"order_status"})
order_details_status = order_details_status.rename(columns={"status_name":"order_details_status"})

orders_with_status = pd.merge(orders, orders_status, "left", left_on="order_key", right_on="id")
order_details_with_status = pd.merge(order_details, order_details_status, "left", left_on="status_id", right_on="id")
fact_orders_all = pd.merge(orders_with_status,order_details_with_status, "inner", left_on="order_key", right_on="order_id")

lab2_fact_orders_list = ["order_key",
"employee_key",
"customer_key",
"order_date",
"shipped_date",
"shipper_key",
"ship_name",
"ship_address",
"ship_city",
"ship_state_province",
"ship_zip_postal_code",
"ship_country_region",
"shipping_fee",
"taxes",
"payment_type",
"paid_date",
"tax_rate",
"product_key",
"quantity",
"unit_price",
"discount",
"order_status",
"order_details_status"]

fact_orders = fact_orders_all[lab2_fact_orders_list]


In [13]:
#obs: the primary key was set before so I used update
set_dataframe(fact_orders,dst_db_name, "fact_orders", "update","fact_order_key")

In [14]:
#validate insertion
#readSqlQuery("SELECT * FROM fact_orders", dst_db_name).head(2)